### Task 3

<span style="color: green; font-size: 14px;">
В этом упражнении ты создашь так называемую витрину данных. Она представляет собой таблицу, 
которую можно использовать для аналитических целей. Обычно она создается путем объединения нескольких отдельных таблиц. 
В этом упражнении мы будем собирать различные данные о наших пользователях: когда они сделали свои первые коммиты, 
когда они впервые посетили ленту новостей и т. д. Это поможет позднее выполнить анализ данных.

Что тебе нужно сделать в этом упражнении (ознакомься с полным описанием задания):
</span>
<span style="color: green; font-size: 13px;">

1. Создай соединение с базой данных с помощью библиотеки `sqlite3`.
</span>

In [2]:
import sqlite3

# cоздаём соединение
conn = sqlite3.connect('data/checking-logs.sqlite')

<span style="color: green; font-size: 13px;">

2. Создай новую таблицу `datamart` в базе данных, объединив таблицы `pageviews` и `checker` с помощью только **одного** запроса.
    - Таблица должна содержать следующие столбцы: `uid`, `labname`, `first_commit_ts`, `first_view_ts`.
    - `first_commit_ts` — это просто новое имя для столбца `timestamp` из таблицы `checker`; он показывает первый коммит конкретного лабораторного задания конкретного пользователя.
    - `first_view_ts` — первое посещение пользователем из таблицы `pageviews`, метка времени посещения пользователем ленты новостей.
    - По-прежнему нужно использовать фильтр `status = 'ready'`.
    - По-прежнему нужно использовать фильтр `numTrials = 1`.
    - Имена лабораторных заданий по-прежнему должны быть из следующего списка: `laba04`, `laba04s`, `laba05`, `laba06`, `laba06s`, `project1`.
    - Таблица должна содержать только пользователей (`uid` с `user_*`), а не администраторов.
    - `first_commit_ts` и `first_view_ts` должны быть распарсены как `datetime64[ns]`.
</span>

In [3]:
import pandas as pd

# cоздаём новую таблицу datamart
query = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT
    checker.uid AS uid,
    checker.labname AS labname,
    datetime(checker.timestamp) AS first_commit_ts,
    datetime(pageviews.datetime) AS first_view_ts
FROM
    checker
LEFT JOIN
    pageviews
ON
    checker.uid = pageviews.uid
WHERE
    checker.status = 'ready'
    AND checker.numTrials = 1
    AND checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY
    checker.uid, checker.labname;
"""
conn.execute(query)

datamart = pd.io.sql.read_sql("SELECT * FROM datamart", conn, parse_dates=['first_commit_ts', 'first_view_ts'])
datamart.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18,2020-04-26 21:53:59
1,user_1,laba04s,2020-04-26 17:12:11,2020-04-26 21:53:59
2,user_1,laba05,2020-05-02 19:15:18,2020-04-26 21:53:59
3,user_1,laba06,2020-05-17 16:26:35,2020-04-26 21:53:59
4,user_1,laba06s,2020-05-20 12:23:37,2020-04-26 21:53:59


<span style="color: green; font-size: 13px;">
    
3. Используя методы библиотеки Pandas, создай два датафрейма: `test` и `control`.
    - `test` должен включать пользователей, у которых имеются значения в `first_view_ts`.
    - `control` должен включать пользователей, у которых отсутствуют значения в `first_view_ts`.
    - Замени пропущенные значения в `control` средним значением `first_view_ts` пользователей из `test` (оно пригодится нам для анализа в будущем).
    - Сохрани обе таблицы в базе данных (вы будете использовать их в следующих упражнениях).
</span>

In [4]:
# cоздаём датафрейм test с пользователями, у которых есть значения в first_view_ts
test = datamart[datamart['first_view_ts'].notnull()]

# cоздаём датафрейм control с пользователями, у которых отсутствуют значения в first_view_ts
control = datamart[datamart['first_view_ts'].isnull()]

# заполняем пропущенные значения в столбце first_view_ts датафрейма control средним значением из test
mean_value = test['first_view_ts'].mean()
control.loc[:, 'first_view_ts'] = control['first_view_ts'].fillna(mean_value)

# сохраняем датафреймы test и control в базе данных
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

test.head()

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18,2020-04-26 21:53:59
1,user_1,laba04s,2020-04-26 17:12:11,2020-04-26 21:53:59
2,user_1,laba05,2020-05-02 19:15:18,2020-04-26 21:53:59
3,user_1,laba06,2020-05-17 16:26:35,2020-04-26 21:53:59
4,user_1,laba06s,2020-05-20 12:23:37,2020-04-26 21:53:59


In [5]:
control.head()

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55,2020-04-27 00:40:05.322033664
13,user_11,project1,2020-05-03 23:45:33,2020-04-27 00:40:05.322033664
14,user_12,laba04,2020-04-18 17:07:51,2020-04-27 00:40:05.322033664
15,user_12,laba04s,2020-04-26 15:42:38,2020-04-27 00:40:05.322033664
16,user_12,laba05,2020-05-03 08:39:25,2020-04-27 00:40:05.322033664


<span style="color: green; font-size: 13px;">

4. Закрой соединение.
</span>
<span style="color: green; font-size: 12px;">

Небольшой совет — выполняй все операции поочередно, от простой к более сложной, а не пытаясь сделать всё вместе и сразу. Это поможет в отладке твоих запросов.
</span>

In [6]:
# закрываем соединение
conn.close()